In [ ]:
def mri_pipeline(subject):
    import numpy as np
    import os.path as op
    import AFQ.data as afd
    import nibabel as nib
    import dipy.reconst.dti as dti
    from dipy.core.gradients import gradient_table    
    from dipy.segment.mask import median_otsu
    from dipy.denoise import nlmeans
    from dipy.denoise.noise_estimate import estimate_sigma
    # This grabs the data from HCP:
    print("Fetching HCP data")
    data_dict = afd.fetch_hcp([int(subject)])

    home = op.expanduser('~')
    gtab = gradient_table(
        op.join(home, 'AFQ_data/HCP/sub-%d/sess-01/dwi/sub-%d_dwi.bval'%(subject, subject)), 
        op.join(home, 'AFQ_data/HCP/sub-%d/sess-01/dwi/sub-%d_dwi.bvec'%(subject, subject)),
        b0_threshold=10)
    
    print("Loading data from disk")
    img = nib.load(
        op.join(home, 'AFQ_data/HCP/sub-%d/sess-01/dwi/sub-%d_dwi.nii.gz'%(subject, subject)))
    data = img.get_data()
    b0 = np.mean(data[..., gtab.b0s_mask], -1)
    print("Generating the mask")
    if not op.exists('./mask.nii.gz'):
        _, mask = median_otsu(b0, 4, 2, False, dilate=1)
        nib.save(nib.Nifti1Image(mask.astype(int), img.affine), './mask.nii.gz')

    # Denoise
    print("Denoising")
    if not op.exists('./denoised_data.nii.gz'):
        sigma = estimate_sigma(data)
        denoised_data = nlmeans.nlmeans(data, sigma=sigma, mask=mask)
        nib.save(nib.Nifti1Image(denoised_data, img.affine), './denoised_data.nii.gz')
    
    #Tensor model
    print("Fitting the Tensor model")
    ten_model = dti.TensorModel(gtab)
    ten_fit = ten_model.fit(denoised_data, mask=mask)
    nib.save(nib.Nifti1Image(ten_fit.fa, img.affine), 'dti_fa.nii.gz')
    nib.save(nib.Nifti1Image(ten_fit.md, img.affine), 'dti_md.nii.gz')
    return ten_fit.fa

In [2]:
import numpy as np

In [68]:
subjects = np.loadtxt('./subjectids.txt')

In [69]:
len(subjects)

25

In [70]:
import cloudknot as ck

In [6]:
github_installs=("https://github.com/yeatmanlab/pyAFQ.git")
my_image = ck.DockerImage(func=mri_pipeline, 
                          base_image='cloudknot/mri_pipeline:hcp', 
                          github_installs=github_installs)


In [7]:
knot = ck.Knot(name="benchmark_mri_pipeline", 
               docker_image=my_image,
               resource_type = "SPOT",
               memory = 32000,
               bid_percentage=100)

In [71]:
results = knot.map(subjects)

In [72]:
jj = knot.jobs[0]

In [73]:
jj.status

{'status': 'FAILED',
 'statusReason': 'Array Child Job failed',
 'attempts': [],
 'arrayProperties': {'statusSummary': {'STARTING': 0,
   'FAILED': 1,
   'RUNNING': 0,
   'SUCCEEDED': 24,
   'RUNNABLE': 0,
   'SUBMITTED': 0,
   'PENDING': 0},
  'size': 25}}

In [11]:
# knot.clobber()

In [12]:
# results.exception()